In [ ]:
#Effective Rainfall 
import xarray as xr
import numpy as np

# Load the NetCDF file
file_path = 'PR_Monthly.nc'
ds = xr.open_dataset(file_path)

# Filter the dataset for the years 2000 to 2021
filtered_ds = ds.sel(time=slice('2000-01-01', '2021-12-31'))

# Extract precipitation data
precipitation = filtered_ds['precipitation']

# Define a function to calculate effective rainfall
def calculate_effective_rainfall(precipitation):
    # Example using a simple empirical method
    pe = np.where(precipitation <= 250, precipitation * (125 - 0.2 * precipitation) / 125, 125)
    return pe

# Apply the effective rainfall calculation across the dataset
effective_rainfall = calculate_effective_rainfall(precipitation)

# Create a new dataset for the effective rainfall
effective_rainfall_ds = xr.Dataset(
    {
        'effective_rainfall': (['time', 'lat', 'lon'], effective_rainfall)
    },
    coords={
        'time': filtered_ds['time'],
        'lat': ds['lat'],
        'lon': ds['lon']
    }
)

# Save the effective rainfall dataset to a new NetCDF file
output_file_path = '/Effective_Rainfall.nc'
effective_rainfall_ds.to_netcdf(output_file_path)

print("Effective rainfall calculation and saving to NetCDF completed.")


In [ ]:
#IWR calculation （could be used for other crops)
import netCDF4 as nc
import numpy as np
from scipy.interpolate import RegularGridInterpolator

# Load the effective rainfall data
er_file = '/Effective_Rainfall.nc'
er_data = nc.Dataset(er_file, mode='r')
er = er_data.variables['effective_rainfall'][:]
lon_er = er_data.variables['lon'][:]
lat_er = er_data.variables['lat'][:]
time_er = er_data.variables['time'][:]

# Load the crop water requirement data
cwr_file = '/CWR_wheat_global.nc' #change the CWR for maize or rice etc
cwr_data = nc.Dataset(cwr_file, mode='r')
cwr = cwr_data.variables['CWR_wheat'][:]
lon_cwr = cwr_data.variables['lon'][:]
lat_cwr = cwr_data.variables['lat'][:]
time_cwr = cwr_data.variables['time'][:]

# Ensure time alignment (2000-2021 for ER and 2000-2022 for CWR)
start_year = 2000
end_year_er = 2021
end_year_cwr = 2022
months_per_year = 12
total_months_er = (end_year_er - start_year + 1) * months_per_year
total_months_cwr = (end_year_cwr - start_year + 1) * months_per_year

if time_er.shape[0] != total_months_er:
    raise ValueError("Time dimensions for ER data do not match the expected range (2000-2021).")

if time_cwr.shape[0] != total_months_cwr:
    raise ValueError("Time dimensions for CWR data do not match the expected range (2000-2022).")

# Truncate the CWR data if it extends beyond 2021
cwr = cwr[:total_months_er, :, :]

# Interpolate ER to match CWR dimensions
lon_new, lat_new = np.meshgrid(lon_cwr, lat_cwr)

def interpolate_er_to_cwr(er, lon_er, lat_er, lon_new, lat_new):
    interpolated_er = np.zeros((er.shape[0], lat_new.shape[0], lon_new.shape[1]))
    for t in range(er.shape[0]):
        interpolator = RegularGridInterpolator((lat_er, lon_er), er[t, :, :], bounds_error=False, fill_value=None)
        points = np.array([lat_new.ravel(), lon_new.ravel()]).T
        interpolated_er[t, :, :] = interpolator(points).reshape(lat_new.shape)
    return interpolated_er

er_interp = interpolate_er_to_cwr(er, lon_er, lat_er, lon_new, lat_new)

# Initialize an array to store the IWR
iwr = np.maximum(0, cwr - er_interp)

# Define a new netCDF file to store the IWR
output_file = '/IWR_wheat.nc'
with nc.Dataset(output_file, 'w', format='NETCDF4') as dst:
    # Create dimensions
    dst.createDimension('time', total_months_er)
    dst.createDimension('lat', lat_cwr.shape[0])
    dst.createDimension('lon', lon_cwr.shape[0])
    
    # Create variables
    times = dst.createVariable('time', 'f4', ('time',))
    latitudes = dst.createVariable('lat', 'f4', ('lat',))
    longitudes = dst.createVariable('lon', 'f4', ('lon',))
    iwr_var = dst.createVariable('IWR_wheat', 'f4', ('time', 'lat', 'lon',))
    
    # Assign attributes
    times.units = 'days since 2000-01-31'
    times.calendar = 'proleptic_gregorian'
    latitudes.units = 'degrees_north'
    longitudes.units = 'degrees_east'
    iwr_var.units = 'mm/month'
    iwr_var.long_name = 'Irrigation Water Requirement for wheat'
    
    # Write data
    times[:] = time_er[:]  # Note: Use time_er since CWR is truncated
    latitudes[:] = lat_cwr[:]
    longitudes[:] = lon_cwr[:]
    iwr_var[:] = iwr

print("Irrigation Water Requirement calculation and file creation completed successfully.")
